In [1]:
import mysql.connector
import redis
import json
import numpy as np

# MySQL connection details
MYSQL_HOST = 'localhost'
MYSQL_USER = 'root'
MYSQL_PASSWORD = ''
MYSQL_DB = 'stock_data'

# Redis connection
REDIS_HOST = 'localhost'
REDIS_PORT = 6379


In [2]:
# Function to connect to MySQL
def connect_mysql():
    return mysql.connector.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        database=MYSQL_DB
    )

# Function to store data in MySQL
def store_stock_data_in_mysql(historical_data, predicted_data_lstm, predicted_data_rf, dates, tech_list):
    conn = connect_mysql()
    cursor = conn.cursor()

    for ticker in tech_list:
        for date, historical_price in zip(dates, historical_data[ticker]):
            predicted_lstm = predicted_data_lstm[ticker][list(dates).index(date)]
            predicted_rf = predicted_data_rf[ticker][list(dates).index(date)]
            
            cursor.execute("""
                INSERT INTO stock_prices (ticker, date, historical_price, predicted_price_lstm, predicted_price_rf)
                VALUES (%s, %s, %s, %s, %s)
            """, (ticker, date, historical_price, predicted_lstm, predicted_rf))

    conn.commit()
    cursor.close()
    conn.close()


In [3]:
# Function to connect to Redis
def connect_redis():
    return redis.StrictRedis(host=REDIS_HOST, port=REDIS_PORT, db=0)

# Function to store real-time data in Redis
def store_real_time_data_in_redis(ticker, real_time_data):
    redis_client = connect_redis()
    redis_client.set(ticker, json.dumps(real_time_data))

# Function to get real-time data from Redis
def get_real_time_data_from_redis(ticker):
    redis_client = connect_redis()
    cached_data = redis_client.get(ticker)
    
    if cached_data:
        return json.loads(cached_data)
    else:
        return None  # Cache miss
